# Conditional Circuits

Let's assume we want to parameterize a circuit by means of a neural network, i.e., build and learn a _conditional circuit_. We can do so in cirkit in three steps:
1. we instantiate the symbolic circuit we want to parameterize;
2. we call a functional that takes the symbolic circuit and returns another one that contains the additional information for the parameterization we want;
3. we compile the symbolic circuit by firstly registering the parameterization to the compiler.

We start by instantiating a symbolic circuit on MNSIT images.

In [1]:
from cirkit.templates import data_modalities, utils

symbolic_circuit = data_modalities.image_data(
    (1, 28, 28),                 # The shape of MNIST image, i.e., (num_channels, image_height, image_width)
    region_graph='quad-tree-4',  # Select the structure of the circuit to follow the QuadTree-4 region graph
    input_layer='categorical',   # Use Categorical distributions for the pixel values (0-255) as input layers
    num_input_units=64,          # Each input layer consists of 64 Categorical input units
    sum_product_layer='cp',      # Use CP sum-product layers, i.e., alternate dense layers with Hadamard product layers
    num_sum_units=64,            # Each dense sum layer consists of 64 sum units
)

Note that we did not specify any parameterization for the sum layer parameters and the logits of the Categorical input layers.

Then, we call the functional ```cirkit.symbolic.functional.model_parameterize``` to obtain another symbolic circuit that stores the additional information on how we want to parameterize it.

In [2]:
import cirkit.symbolic.functional as SF
from cirkit.symbolic.layers import CategoricalLayer, SumLayer

parametrization_map = {
    "sum-layers": list(symbolic_circuit.sum_layers)
}

# Assume there exists a model called "my-neural-network" we will specify at compile-time later
symbolic_conditional_circuit, model_specs = SF.model_parameterize(
    symbolic_circuit,                          # The symbolic circut we want to parameterize
    gate_functions=parametrization_map
)

The ```model_parameterize``` functional also returns the shapes of the tensors to parameterize.

In [3]:
# The parameterize function returned the shape specfication of the tensors we will need to return
model_specs

{'g0.SumLayer.weight': (1048, 64, 64), 'g1.SumLayer.weight': (1, 1, 64)}

Before compiling our conditional circuit, we define a neural network in PyTorch.

In [4]:
import torch
from torch import Tensor, nn

class MyNeuralNetwork(nn.Module):
    def __init__(self, output_specs: dict[str, tuple[int, ...]]):
        super().__init__()
        self._output_specs = output_specs

    def forward(self, z: Tensor) -> dict[str, Tensor]:
        # Evaluate my neural network on some input y
        # Return a dictionary mapping parameter names to actual parameter tensors
        # In this example, we just sample them randomly and use a softmax activation
        mean, stddev = torch.mean(z.flatten()), torch.std(z.flatten())
        return {
            name: torch.softmax(mean + stddev * torch.randn(*shape), dim=-1)
            for name, shape in self._output_specs.items()
        }

In [5]:
# Allocate our neural network
my_neural_net = MyNeuralNetwork(model_specs)

Next, we tell the compiler about our model, by specifying the same model name we gave to the ```model_parameterize``` functional.

In [6]:
from cirkit.pipeline import PipelineContext

# Initialize an pipeline compilation context
# Let's try _without_ folding first
ctx = PipelineContext(semiring="lse-sum", backend='torch', fold=False, optimize=False)

# Register our neural network as an external model
ctx.add_gate_function("sum-layers", my_neural_net)

# Finally, we compile the conditional circuit
circuit = ctx.compile(symbolic_conditional_circuit)

Now, to evaluate the conditional circuit we specify additional arguments when calling it on some tensor, as shown below.

In [13]:
batch_size = 8
x = torch.randint(256, size=(batch_size, 1, 784))  # The circuit input
# TODO: make the parameters batch-dependant
z = torch.randn(size=(1, 127))  # Some dummy input to the neural net

# Evaluate the circuit on some input
# Note that we also pass some input to the external model
circuit(x, gate_function_kwargs={'sum-layers': {'z': z}})

tensor([[[-4356.2993]],

        [[-4353.8691]],

        [[-4364.9561]],

        [[-4356.7891]],

        [[-4359.1729]],

        [[-4361.8467]],

        [[-4355.8101]],

        [[-4358.1465]]], grad_fn=<TransposeBackward0>)

The above parameterization is robust to change in compilation flages, e.g., now enabling folding and layer optimizations.

In [16]:
from cirkit.pipeline import PipelineContext

# Initialize an pipeline compilation context
# Let's try _without_ folding first
ctx = PipelineContext(semiring="lse-sum", backend='torch', fold=True, optimize=True)

# Register our neural network as an external model
ctx.add_gate_function("sum-layers", my_neural_net)

# Finally, we compile the conditional circuit
circuit = ctx.compile(symbolic_conditional_circuit)

In [17]:
batch_size = 8
x = torch.randint(256, size=(batch_size, 1, 784))  # The circuit input
# TODO: make the parameters batch-dependant
z = torch.randn(size=(1, 127))  # Some dummy input to the neural net

# Evaluate the circuit on some input
# Note that we also pass some input to the external model
circuit(x, gate_function_kwargs={'sum-layers': {'z': z}})

tensor([[[-4356.0820]],

        [[-4351.3091]],

        [[-4356.7505]],

        [[-4362.7915]],

        [[-4359.5459]],

        [[-4352.6587]],

        [[-4363.6162]],

        [[-4354.9424]]], grad_fn=<TransposeBackward0>)